#LGG and HGG tumor classification
in this notebook, dataset of BraTS-19 and 18 was taken for classification of HGG and LGG tumors. The dataset has been splitted into training and validation. A function has been written to split data into training and validation, in addition, the dataset has been labeled. HGG=1, and LGG=0
then at end end counting entries function has been written to count entries of each components within training and validation. Finally, both dataset 19, and 18 are merged together to make a larger set for training and validation. Training set has almost 80% data and validation set has more and less 20% of total data.

In [199]:
# importing necessary packages
import os
import sys
import glob
from google.colab import drive
import random
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [200]:
# writing custom function to prepare our dataset
def preprocess_split_data(data_path, seed=23):
  ''' preprocess data in data_path folder and split HGG and LGG files except seg.nii.gz
      take equal number of files from both folder and put it in LGG and HGG files
      data_path: path to data folder
      return
      training_data, and validation_data, and data in a dictionary for training'''
  random.seed(seed)
  LGG_path = os.path.join(data_path, 'LGG') 
  HGG_path = os.path.join(data_path, 'HGG')
  if len(os.listdir(HGG_path)) > len(os.listdir(LGG_path)):
    HGG_folders = os.listdir(HGG_path)
    random.shuffle(HGG_folders)
    HGG_folders = HGG_folders[:len(os.listdir(LGG_path))]
  LGG_folders = os.listdir(LGG_path)
  random.shuffle(LGG_folders)
  if len(LGG_folders) == len(HGG_folders):
    print('Both folders have now equal length')
    lgg_folders_paths = []
    hgg_folders_paths = []
    for folder in LGG_folders:
      folder_path = os.path.join(LGG_path, folder)
      lgg_folders_paths.append(folder_path)
    for folder in HGG_folders:
      folder_path = os.path.join(HGG_path, folder)
      hgg_folders_paths.append(folder_path)
  lgg_files = []
  hgg_files = []
  for i in range(len(lgg_folders_paths)):
    files = sorted(glob.glob(os.path.join(lgg_folders_paths[i], '*.nii.gz')))
    lgg_files.append(files)
  for j in range(len(hgg_folders_paths)):
    files = sorted(glob.glob(os.path.join(hgg_folders_paths[j], '*.nii.gz')))
    hgg_files.append(files)
  LGG_files = []
  HGG_files = []
  for i in range(len(lgg_files)):
    for j in range(5):
      l_fils = lgg_files[i][j]
      LGG_files.append(l_fils)
  for file in LGG_files:
    if 'seg.nii.gz' in file:
      LGG_files.remove(file)
    else:
      pass
  for i in range(len(hgg_files)):
    for j in range(5):
      h_fils = hgg_files[i][j]
      HGG_files.append(h_fils)
  for file in HGG_files:
    if 'seg.nii.gz' in file:
      HGG_files.remove(file)
    else:
      pass
  data_paths = HGG_files + LGG_files
  random.shuffle(data_paths)
  lbl = 1 
  dataset_paths = [{'image': img, 'label':lbl} for img in data_paths]
  for index in range(len(dataset_paths)):
    if 'HGG' in dataset_paths[index]['image']:
        dataset_paths[index]['label']=1
    else:
        dataset_paths[index]['label']=0
  train_size = 0.8
  train_data = dataset_paths[:int(len(dataset_paths)*train_size)]
  val_data = dataset_paths[int(len(dataset_paths)*train_size):]
  return train_data, val_data, dataset_paths
        

In [201]:
data_path = "/gdrive/MyDrive/Medical Dataset/Brats'18'"
seed = 111

In [202]:
def count_entries(data):
  count_flair=0
  count_t1 =0
  count_t1ce =0
  count_t2 =0
  for file in data:
    if 'flair.nii.gz' in file['image']:
      count_flair+=1
    elif 't1ce.nii.gz' in file['image']:
      count_t1ce+=1
    elif 't1.nii.gz' in file['image']:
      count_t1 +=1
    else:
      count_t2+=1
  LGG_count=0
  HGG_count=0
  for file in data:
    if 'LGG' in file['image']:
      LGG_count+=1
    else:
      HGG_count+=1
  return (LGG_count, HGG_count, count_flair, count_t1, count_t2, count_t1ce)

In [203]:
data_path1 = '/gdrive/MyDrive/Medical Dataset/MICCAI_BraTS_2019_Data_Training'

In [204]:
seed1= 222


In [205]:
def ready_all_data(data_path, data_path1):
  ''' this funtion ready all the dataset into training and validation sets for trainiing
      data_path: data_path of 2019 Brats dataset
      data_path1: data path of 2018 Brats dataset
      return 
      training_data: shuffled training data (80%)
      validation_data: shuffled validatoin data (20%)
      '''
  train, val, data = preprocess_split_data(data_path, seed)
  train1, val1, data1 = preprocess_split_data(data_path1, seed1)
  training_data = train + train1
  validation_data = val + val1
  all_data = data + data1
  random.seed(11)
  random.shuffle(training_data)
  random.shuffle(validation_data)
  random.shuffle(all_data)
  return (training_data, validation_data, all_data)



In [206]:
training_data, validation_data, _ = ready_all_data(data_path, data_path1)

Both folders have now equal length
Both folders have now equal length


In [207]:
len(training_data)

966

In [208]:
LGG_count, HGG_count, count_flair, count_t1, count_t2, count_t1ce = count_entries(validation_data)

In [209]:
print(f'counting differnting classes entry in validation data with seed:{seed}')
print(f'number of flairs: {count_flair}')
print(f'number of t1: {count_t1}')
print(f'number of t2: {count_t2}')
print(f'number of t1ce: {count_t1ce}')
print(f'LGG count: {LGG_count}')
print(f'HGG count: {HGG_count}')

counting differnting classes entry in validation data with seed:111
number of flairs: 61
number of t1: 55
number of t2: 67
number of t1ce: 59
LGG count: 121
HGG count: 121
